In [ ]:
#Works

In [1]:
# Image/Video reading and recognition
import cv2
# Vector Calculations
import numpy as np
# displaying Video (Deprecated)
from IPython.display import display, Image
# Stop Button
import ipywidgets as widgets
# For Showing Video
import threading
#Delays
import time
#Connecting to the Jetbot
from jetbot import Robot


In [2]:
robot = Robot()

In [27]:
#Creating a Stop Button
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop', #What it will say
    disabled=False, # ENABLE
    button_style='danger', #RED
    tooltip='Description', #
    icon='square' #Box Button
)
########################################

#Identifies features of a human face
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Display function
# ================

def view(button):
    """
    This function uses the HAAR Cascade detection method to detect a face and keep it in the center of the frame
    button: A button object that will perform some command (Close the video/stop bot)
    """
    #Displays Video
    cap = cv2.VideoCapture("nvarguscamerasrc ! video/x-raw(memory:NVMM),width=640,height=480,framerate=30/1,format=NV12 ! nvvidconv ! video/x-raw,format=BGRx ! videoconvert ! video/x-raw,format=BGR ! appsink drop=1", cv2.CAP_GSTREAMER)
    display_handle=display(None, display_id=True)
    i = 0
    
    #Initialize necessary variables for movement
    leftcount=0
    rightcount=0
    forwardscount=0
    backwardscount=0
    rightdistance=[]
    leftdistance=[]
    for_distance=[]
    back_distance=[]

    while True:
        init = time.time()
        #Gets the Image from camera
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        #changes from BGR to Grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #Calls the Face Classifier object to identify faces in the captured image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3,
    minNeighbors=5,
    #Minimum size of box                                          
    minSize=(20, 20))
        
        if faces != ():
            #Obtain the top left and width/height of the box
            (x, y, w, h) = faces[0]
            #Forms a rectangle around the identified face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

##########################################
#######turning left and right######################
            #set center of picture
            x0=320
            #if the difference of the frame (x) and x origin is smaller than -30 (change according to camera)
            #the car should turn right
            if x-x0<-30:
                direction='right'
                rightcount=rightcount+1
                rightdistance.append(x-x0)
            #if the difference of the frame (x) and x origin is greater than 30 (change according to camera)
            #the car should turn left
            if x-x0>30:
                direction='left'
                leftcount=leftcount+1
                leftdistance.append(x-x0)
            #Hyperparameters to adjust movement time
            div = 800
            div2 = 30
            fb_speed =.3
            t_speed = .3
            cnt = 2
            #If We find the box on the left cnt times -> run
            if leftcount>cnt:
                leftlevel=np.abs(np.mean(leftdistance))
                robot.backward(t_speed)
                time.sleep(leftlevel/div)
                robot.stop()
                leftcount=0
                rightcount=0
                rightdistance=[]
                leftdistance=[]
                print('left: ',leftlevel)
            #If We find the box on the right cnt times -> run
            if rightcount>cnt:
                rightlevel=np.abs(np.mean(rightdistance))
                robot.forward(t_speed)
                time.sleep(rightlevel/div)
                robot.stop()
                rightcount=0
                leftcount=0
                rightdistance=[]
                leftdistance=[]
                print('right: ',rightlevel)

########################################
#######turning forward and backward######################
        
            w0=90
            # Check if the box is below/above threshold
            if w-w0>10:
                #backwards=left
                direction='backwards'
                backwardscount=backwardscount+1
                back_distance.append(w-w0)
            if w-w0<-10:
                direction='forwards'
                forwardscount=forwardscount+1
                for_distance.append(w-w0)
            # Check if threshold is broken cnt times 
            if backwardscount>cnt:
                backlevel=np.abs(np.mean(back_distance)) #Average of found boxes 
                robot.left(fb_speed)
                time.sleep(backlevel/div2) # Distance from center determines how long we will move
                robot.stop()
                 #re-initialize
                backwardscount=0 
                forwardscount=0
                forwardsdistance=[]
                backwardsdistance=[]
            if forwardscount>cnt:
                forlevel=np.abs(np.mean(for_distance))
                robot.right(fb_speed) 
                time.sleep(forlevel/div2) # Distance from center determines how long we will move
                robot.stop()
                 #re-initialize
                backwardscount=0
                forwardscount=0
                forwardsdistance=[]
                backwardsdistance=[]
            
########################################################
        #Converts Jpeg to Streaming Data (Video)
        _, frame = cv2.imencode('.jpeg', frame)
        #print('Time to Loop:', time.time()- init)
        #Display/Update the video Stream
        display_handle.update(Image(data=frame.tobytes()))
        #If the Stop button is pressed close video and stop robot
        if stopButton.value==True:
            cap.release()
            robot.stop()
            display_handle.update(None)

            
# Run
# ================
#Show the stop button
display(stopButton)
#Create a thread object to call the view fcn that takes the stop btn as a parameter
thread = threading.Thread(target=view, args=(stopButton,))
# Call the start function on the thread object
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


right:  301.6666666666667
right:  292.0
right:  132.66666666666666
right:  54.666666666666664
right:  40.666666666666664
left:  43.333333333333336
right:  83.66666666666667
right:  204.66666666666666
right:  191.0
left:  37.666666666666664
right:  65.66666666666667
right:  120.0
right:  71.66666666666667
right:  53.333333333333336
right:  53.0
right:  52.0
right:  51.0
right:  41.333333333333336
